# Module 12: Images

In this module, we are going to  do an exercise involving images. In the different realms of engineering, images are used everywhere. Photos, satellite images, and aerial photos are all images. 

Images are basically layers of gridded data with an intensity value at each pixel. The layers in the image are called channels(e.g., the red, green and blue channels), and the combination of these channels produces a specific color at each pixel.

The following example illustrates the loading, manipulation and saving of images.

But first, you need to install some stuff...

## What you need to install
In Python, we can use the pillow library to load, display and manipulate images. We believe pillow is installed in the Anaconda distribution, but if not, you will need to install it as follows:

In [ ]:
# run this cell if pillow is not installed
import sys
!{sys.executable} -m pip install pillow

### Importing an image of your choice
We are importing our image: "SURF.JPG". OBS: You have to make sure the image you choose to import is in the same folder as your notebook.

In [ ]:
# import and print image

from PIL import Image
import os

im = Image.open(os.path.join("SURF.JPG")) # import image of your choice in "SURF.JPG"

print(im.format, im.size, im.mode) # print image information

im

The image is in *portable network graphics* PNG format, and it has 6000 by 3376 pixels, in three channels: red (R), green (G) and blue (B). Now, let's convert the image to an array using the `numpy.array` method, and print the shape, and maximum and minimum values of the array:

In [ ]:
import numpy as np

data = np.array(im) # convert image to array, "asarray" also works here

print(data.shape) # print shape of array

print("minimum value =", np.amin(data), "maximum value =", np.amax(data)) # print max and min array values

The array is a 3D array, which basically represents 3 (`data.shape[2]`) 2D layered grids (each one 3376 x 6000 pixels in size), corresponding to the R, G and B channels. The minimum and maximum values in the grid are 0 and 255. These values come from the format of the image (JPG), which uses a fixed 8-bit colormap of $2^8=256$ possible combinations representing different intensities. Now, let's plot the image:

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(8, 6)) # create figure

ax.imshow(data); # plot image

Now lets see our color channels in a histogram.

In [ ]:
colors = ["red", "green", "blue"] # list of colors
channel_ids = [0, 1, 2] # list of channels ids

fig, ax = plt.subplots() # create figure

# plot histogram
for channel_id, c in zip(channel_ids, colors): # iterate over each channel ids and colors
    histogram, bin_edges = np.histogram(data[:, :, channel_id], bins=256, range=(0, 256)) # compute histogram
    ax.plot(bin_edges[0:-1], histogram, color=c, label=c + " channel") # plot histogram

# figure title, axes labels and legend
ax.set_title("Histogram Surf image")
ax.set_xlabel("value")
ax.set_ylabel("pixels")
ax.legend(loc = "upper left");

Here are the histograms of the individual  color channels.

Now, let's do something more interesting. Let's process the image to better highlight its main components. To do this, we use the [scikit-image](https://scikit-image.org) library, which is a collection of algorithms for image processing. We believe `scikit-image` is installed in the Anaconda distribution, but if not, you will need to install it as follows:

In [ ]:
# run this cell if scikit-image is not installed
import sys
!{sys.executable} -m pip install scikit-image

Let's convert the image to gray scale using the `scikit-image.color.rgb2gray` function (more about how this function works [here](https://scikit-image.org/docs/dev/auto_examples/color_exposure/plot_rgb_to_gray.html)), and then smooth the image a little bit using a gaussian filter ([sckit-image.filters.gaussian](https://scikit-image.org/docs/stable/api/skimage.filters.html#skimage.filters.gaussian)). We also print the size of the filtered image, the minimum and maximum values, and plot the image:


In [ ]:
from skimage import color, filters

data_gray = color.rgb2gray(data) # converts RGB image to single grayscale channel

data_grad = filters.gaussian(data_gray) # apply Gaussian filter to grayscale image

# print size and minimum and maximum values
print(data_grad.shape) 
print("minimum value = {:.3f}, maximum value = {:.3f}".format(np.amin(data_grad), np.amax(data_grad)) ) 

# plot filtered image. You can also try to use the 'viridis colormap'
fig, ax = plt.subplots(figsize=(8, 6))
plt.imshow(data_grad, cmap='gray')# Using the "gray" colormap for grayscale
cbar= plt.colorbar();# gray scale bar
cbar.set_label('Intensity', rotation=270, labelpad=10)


Now, let´s  try  to blur and sharpen our picture, as well as change brightness and contrast:

In [ ]:
from PIL import Image, ImageEnhance, ImageFilter
from IPython.display import display

def process_and_display(input_path):
    # Open the image
    img = Image.open(input_path)

    # Display the original image
    print("Original Image:")
    display(img)

    # Blur the image
    img_blurred = img.filter(ImageFilter.GaussianBlur(radius=10))#adjust the radius for more blur
    print("Blurred Image:")
    display(img_blurred)

    # Sharpen the image
    #img_sharpened = img.filter(ImageFilter.SHARPEN)
    img_sharpened = img.filter(ImageFilter.UnsharpMask(radius=3, percent=300, threshold=3))
    print("Sharpened Image:")
    display(img_sharpened)

    # Change brightness
    enhancer_brightness = ImageEnhance.Brightness(img)
    img_brightness = enhancer_brightness.enhance(1.5)  # Adjust the factor as needed
    print("Brightened Image:")
    display(img_brightness)

    # Change contrast
    enhancer_contrast = ImageEnhance.Contrast(img)
    img_contrast = enhancer_contrast.enhance(1.5)  # Adjust the factor as needed
    print("High-Contrast Image:")
    display(img_contrast)


# Replace this with your actual input path
input_image_path = os.path.join("SURF.JPG")

# Example usage
process_and_display(input_image_path)